# IBM Coursera Data Science Project

This Notebook will be mainly used for the Capstone Project for the IBM Data Science Specialization via Coursera.

In [1]:
import numpy as np
import pandas as pd

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [3]:
import itertools
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

### About dataset

This dataset is about Collisions, provided by SPD and recorded by Traffic Records. The dataset includes all types of collisions. Collision will display at the intersection or mid-block of a segment from 2004 to present, updated weekly.

### Load Dataset from CSV file

In [4]:
# The code was removed by Watson Studio for sharing.

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [5]:
df.shape

(194673, 38)

## Introduction: Business Understanding

In order to reduce the frequency of car accidents/collisions in a community, this project is developed to predict the severity of an accident given the current weather, road and visibility conditions. This model will be able to predict the severity of accidents/ collisions and alert the drivers to be more careful if the conditions are critical.

## Data Understanding and Data Preparation

Our target variable will be 'SEVERITYCODE', which is used to measure the severity of an accident. Attributes that will be used to measure the severity of an accident are 'WEATHER', 'ROADCOND', and 'LIGHTCOND', which represent the current weather, road condition, and light condition, respectively.

The raw dataset is not ideal for data analysis and the development of the desired algrorithm. First of all, there are many columns that we will not use for this model. Also, most of the features are of type object, when they should be numerical type. Now it's time to do some extracting and converting to the dataset.

### Data extracting and converting
#### Dropping unnecessary columns/attributes

In [6]:
df = df[['SEVERITYCODE','WEATHER','ROADCOND','LIGHTCOND']]
df.head()

,SEVERITYCODE,WEATHER,ROADCOND,LIGHTCOND
0,2,Overcast,Wet,Daylight
1,1,Raining,Wet,Dark - Street Lights On
2,1,Overcast,Dry,Daylight
3,1,Clear,Dry,Daylight
4,2,Raining,Wet,Daylight


#### Checking the data types

In [7]:
df.dtypes

SEVERITYCODE     int64
WEATHER         object
ROADCOND        object
LIGHTCOND       object
dtype: object

In [8]:
df['SEVERITYCODE'].value_counts()

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

#### Balancing data:
We observed that there is a lot more rows that have SEVERITYCODE as "1" than that as "2", hence, we need to balance the data in order to have unbiased results.

In [9]:
from sklearn.utils import resample
df_majority = df[df.SEVERITYCODE==1]
df_minority = df[df.SEVERITYCODE==2]
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=58188,     # to match minority class
                                 random_state=123)
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
df_downsampled['SEVERITYCODE'].value_counts()


2    58188
1    58188
Name: SEVERITYCODE, dtype: int64

In [10]:
df = df_downsampled
df.head()

,SEVERITYCODE,WEATHER,ROADCOND,LIGHTCOND
25055,1,Raining,Wet,Dark - Street Lights On
65280,1,Clear,Dry,Daylight
86292,1,Unknown,Unknown,Unknown
155111,1,Clear,Dry,Daylight
64598,1,Clear,Dry,Daylight


#### Converting data:
The next step is to convert the categorical data to numerical value data. Let's first check out how each each attribute affects the severity code under each condition:

In [22]:
df.groupby(['WEATHER'])['SEVERITYCODE'].value_counts(normalize = True)

WEATHER  SEVERITYCODE
0        1               0.615057
         2               0.384943
1        1               0.500000
         2               0.500000
2        2               0.527478
         1               0.472522
3        2               0.526761
         1               0.473239
4        1               0.714286
         2               0.285714
5        2               0.519484
         1               0.480516
6        2               0.750000
         1               0.250000
7        2               0.542946
         1               0.457054
8        2               0.538462
         1               0.461538
9        1               0.555556
         2               0.444444
10       1               0.639241
         2               0.360759
11       1               0.880893
         2               0.119107
Name: SEVERITYCODE, dtype: float64

In [12]:
df.groupby(['ROADCOND'])['SEVERITYCODE'].value_counts(normalize = True)

ROADCOND        SEVERITYCODE
Dry             2               0.527158
                1               0.472842
Ice             1               0.597345
                2               0.402655
Oil             2               0.615385
                1               0.384615
Other           2               0.518072
                1               0.481928
Sand/Mud/Dirt   1               0.510638
                2               0.489362
Snow/Slush      1               0.683712
                2               0.316288
Standing Water  2               0.526316
                1               0.473684
Unknown         1               0.889950
                2               0.110050
Wet             2               0.536359
                1               0.463641
Name: SEVERITYCODE, dtype: float64

In [13]:
df.groupby(['LIGHTCOND'])['SEVERITYCODE'].value_counts(normalize = True)

LIGHTCOND                 SEVERITYCODE
Dark - No Street Lights   1               0.614319
                          2               0.385681
Dark - Street Lights Off  1               0.550498
                          2               0.449502
Dark - Street Lights On   1               0.503141
                          2               0.496859
Dark - Unknown Lighting   2               0.571429
                          1               0.428571
Dawn                      2               0.536808
                          1               0.463192
Daylight                  2               0.539054
                          1               0.460946
Dusk                      2               0.541203
                          1               0.458797
Other                     1               0.590551
                          2               0.409449
Unknown                   1               0.900198
                          2               0.099802
Name: SEVERITYCODE, dtype: float64

#### Converting

In [18]:
df.WEATHER = pd.Categorical(df.WEATHER).codes
df.ROADCOND = pd.Categorical(df.ROADCOND).codes
df.LIGHTCOND = pd.Categorical(df.LIGHTCOND).codes
df.head()

,SEVERITYCODE,WEATHER,ROADCOND,LIGHTCOND
25055,1,7,9,3
65280,1,2,1,6
86292,1,11,8,9
155111,1,2,1,6
64598,1,2,1,6


In [19]:
df.groupby(['LIGHTCOND'])['SEVERITYCODE'].value_counts(normalize = True)

LIGHTCOND  SEVERITYCODE
0          1               0.617275
           2               0.382725
1          1               0.614319
           2               0.385681
2          1               0.550498
           2               0.449502
3          1               0.503141
           2               0.496859
4          2               0.571429
           1               0.428571
5          2               0.536808
           1               0.463192
6          2               0.539054
           1               0.460946
7          2               0.541203
           1               0.458797
8          1               0.590551
           2               0.409449
9          1               0.900198
           2               0.099802
Name: SEVERITYCODE, dtype: float64

In [20]:
df.groupby(['ROADCOND'])['SEVERITYCODE'].value_counts(normalize = True)

ROADCOND  SEVERITYCODE
0         1               0.616498
          2               0.383502
1         2               0.527158
          1               0.472842
2         1               0.597345
          2               0.402655
3         2               0.615385
          1               0.384615
4         2               0.518072
          1               0.481928
5         1               0.510638
          2               0.489362
6         1               0.683712
          2               0.316288
7         2               0.526316
          1               0.473684
8         1               0.889950
          2               0.110050
9         2               0.536359
          1               0.463641
Name: SEVERITYCODE, dtype: float64

In [21]:
df.groupby(['WEATHER'])['SEVERITYCODE'].value_counts(normalize = True)

WEATHER  SEVERITYCODE
0        1               0.615057
         2               0.384943
1        1               0.500000
         2               0.500000
2        2               0.527478
         1               0.472522
3        2               0.526761
         1               0.473239
4        1               0.714286
         2               0.285714
5        2               0.519484
         1               0.480516
6        2               0.750000
         1               0.250000
7        2               0.542946
         1               0.457054
8        2               0.538462
         1               0.461538
9        1               0.555556
         2               0.444444
10       1               0.639241
         2               0.360759
11       1               0.880893
         2               0.119107
Name: SEVERITYCODE, dtype: float64

#### The following table shows the detail of the encoding:
| Code |WEATHER                | ROADCOND      | LIGHTCOND                |  
|--- |---                      |---            |---                       |
| 1  | Blowing Sand/Dirt       | Dry           | Dark - No Street Lights  |   
| 2  | Clear                   | Ice           | Dark - Street Lights Off |   
| 3  | Fog/Smog/Smoke          | Oil           | Dark - Street Lights On  |   
| 4  | Other                   | Other         | Dark - Unknown Lighting  |   
| 5  | Overcast                | Sand/Mud/Dirt | Dawn                     |   
| 6  | Partly Cloudy           | Snow/Slush    | Daylight                 |  
| 7  | Raining                 | Standing Water| Dusk                     |   
| 8  | Severe Crosswind        | Unknown       | Other                    |   
| 9  | Sleet/Hail/Freezing Rain| Wet           | Unknown                  |
| 10 | Snowing                 | NA            | NA                       |
| 11 | Unknown                 | NA            | NA                       |



#### Checking Missing Values:

In [80]:
df_missing = df.isna()
df_num_missing = df_missing.sum()

In [81]:
df_num_missing

SEVERITYCODE    0
WEATHER         0
ROADCOND        0
LIGHTCOND       0
dtype: int64

#### Next Step: Modeling
Our data is now ready to be fed into machine learning models. We will use the following models:

K-Nearest Neighbor (KNN), which help us predict the severity code of an outcome by finding the most similar to data point within k distance.

A Decision Tree, which will provide a layout of all possible outcomes so we can fully analyze the concequences of a decision.

Logistic Regression: We can use logistic regression because the data set provides two severity code outcomes, resulted in a binary data set, which is perfect for logistic regression.

## Modeling